In [0]:
%run ../utils/common

In [0]:
# Create input widgets
dbutils.widgets.text("job_name", "", "")
dbutils.widgets.text("env", "", "")
dbutils.widgets.text("proc_date", "", "")

In [0]:
job_name = dbutils.widgets.get("job_name")
env = dbutils.widgets.get("env")
proc_date = dbutils.widgets.get("proc_date")  # Expect format: 'yyyy-MM-dd'

print(job_name)
print(env)
print(proc_date)

In [0]:
spark.sql(f"""
update udp_wcm_dev.default.ref_dag_dependencies
SET dep_cron_pattern = '5 4 * * *'
WHERE pipeline = 'dwh_dim_fact_udp_pipeline' """)

In [0]:
catalog = settings[env]["catalog_name"]
catalog

In [0]:
df = spark.sql(f"""
  SELECT *
  FROM (SELECT * FROM {catalog_name}.default.ref_dag_dependencies WHERE pipeline = '{job_name}') AS dep
  JOIN {catalog_name}.default.meta_job_runs AS job_runs
  ON dep.depends_on = job_runs.job_name 
""")

display(df)

In [0]:
from datetime import datetime

proc_date = {proc_date}

# Parse full datetime
proc_date = datetime.fromisoformat(proc_date)

# Get only the date
proc_date = proc_date.date()

print(proc_date)  # Output: 2025-05-27

# # Parse full datetime
# proc_date = datetime.fromisoformat(proc_date)

# # Get only the date
# date_only = proc_date.date()





In [0]:
proc_ts = datetime.strptime(proc_date, '%Y-%m-%d')
proc_dt = proc_ts.date()
proc_dt

In [0]:
rows = df.collect()

for row in rows:
    cron_schedule = classify_cron_job(row.dep_cron_pattern)
    print(f"cron_schedule: {cron_schedule}")

    if cron_schedule == "Daily":
        expected_run_dt = proc_ts.date() - timedelta(days=1)
        actual_run_dt = row.proc_time.date()

        if actual_run_dt >= expected_run_dt:
            raise ValueError(f"{row.depends_on} did not run on {expected_run_dt}")
        else:
            print(f"{row.depends_on} passed, last run: {actual_run_dt}")

    elif cron_schedule == "Hourly":
        cron = croniter(row.dep_cron_pattern, proc_ts)
        expected_run_ts = cron.get_prev(datetime)
        expected_run_ts = expected_run_ts.replace(second=0, microsecond=0)

        actual_run_ts = row.proc_time.replace(second=0, microsecond=0)

        if expected_run_ts != actual_run_ts:
            raise ValueError(f"{row.depends_on} did not run on {expected_run_ts}")
        else:
            print(f"{row.depends_on} passed: last run: {actual_run_ts}")

    else:
        raise ValueError("No implementation for schedule other than Daily, Hourly")

print(f"All dependencies passed for job: {job_name}")